In [1]:
import caffe
from dataLoader import fc7FrameSequenceGenerator
from hdf5_npstreamsequence_generator import HDF5SequenceWriter
from fileWriter import *
from utils import *

In [2]:
MAX_WORDS = 160
BUFFER_SIZE = 1 # TEXT streams
BATCH_STREAM_LENGTH = 1000

SETTING = '/data/gengshan/pose_s2vt'
FRAMEFEAT_FILE_PATTERN = SETTING + '/splits/dataCsv_{0}.txt'
SENTS_FILE_PATTERN = SETTING + '/splits/dataTsv_{0}.txt'  # input paths

OUTPUT_DIR = '{0}/hdf5/buffer_{1}_s2vt_{2}'.format(SETTING, BUFFER_SIZE, MAX_WORDS)
OUTPUT_DIR_PATTERN = '%s/%%s_batches' % OUTPUT_DIR
VOCAB = '%s/vocabulary.txt' % SETTING

OUT_FILE_PATTERN = SETTING + '/rawcorpus/{0}/s2vt_vgg_{0}_sequence.txt'
OUT_CORPUS_PATH = SETTING + '/rawcorpus/{0}'

LSTM_NET_FILE = './s2vt.words_to_preds.deploy.prototxt'
RESULTS_DIR = './results'
MODEL_FILE = './snapshots/s2vt_youtube_vgg_iter_17000.caffemodel'
SENTS_FILE = None
NET_TAG = 's2vt_youtube_vgg_iter_17000'

In [3]:
caffe.set_mode_gpu()
caffe.set_device(0)
lstm_net = caffe.Net(LSTM_NET_FILE, MODEL_FILE, caffe.TEST)

STRATEGIES = [{'type': 'beam', 'beam_size': 1}]
NUM_OUT_PER_CHUNK = 30
START_CHUNK = 0
vocab_file = '/data/gengshan/s2vt/vocabulary_whole.txt'

DATASETS = [('HAHAHAHA', 'haha', True)] # split_name, data_split_name, aligned

for split_name, data_split_name, aligned in DATASETS:
    filenames =  [FRAMEFEAT_FILE_PATTERN.format(DATASETS[0][1]),
               SENTS_FILE_PATTERN.format(DATASETS[0][1])]

    fsg = fc7FrameSequenceGenerator(filenames, BUFFER_SIZE,
          vocab_file, max_words=MAX_WORDS, align=aligned, shuffle=False,
          pad=aligned, truncate=aligned)
    
    
    video_gt_pairs = all_video_gt_pairs(fsg)
    print 'Read %d videos pool feats' % len(fsg.vid_framefeats)
    NUM_CHUNKS = (len(fsg.vid_framefeats)/NUM_OUT_PER_CHUNK) + 1
    eos_string = '<EOS>'
    # add english inverted vocab 
    vocab_list = [eos_string] + fsg.vocabulary_inverted
    offset = 0
    
    
    for c in range(START_CHUNK, NUM_CHUNKS):
        chunk_start = c * NUM_OUT_PER_CHUNK
        chunk_end = (c + 1) * NUM_OUT_PER_CHUNK
        chunk = video_gt_pairs.keys()[chunk_start:chunk_end]
        html_out_filename = '%s/%s.%s.%d_to_%d.html' % \
          (RESULTS_DIR, data_split_name, NET_TAG, chunk_start, chunk_end)
        text_out_filename = '%s/%s.%s_' % \
          (RESULTS_DIR, data_split_name, NET_TAG)

        if not os.path.exists(RESULTS_DIR): os.makedirs(RESULTS_DIR)
        outputs = run_pred_iters(lstm_net, chunk, video_gt_pairs,
                    fsg, strategies=STRATEGIES, display_vocab=vocab_list)

        html_out = to_html_output(outputs, vocab_list)
        html_out_file = open(html_out_filename, 'w')
        html_out_file.write(html_out)
        html_out_file.close() 
            

Reading frame features from file: /data/gengshan/pose_s2vt/splits/dataCsv_haha.txt
Reading sentences in: /data/gengshan/pose_s2vt/splits/dataTsv_haha.txt
Initializing the vocabulary.
Initialized vocabulary from file with 6067 unique words (from 6066 total words in dataset).
Processed 0/608 (0.000000%) lines
Found 1 videos with 1 captions
Processed 1/608 (0.164474%) lines
Found 2 videos with 2 captions
Found 3 videos with 3 captions
Found 4 videos with 4 captions
Found 5 videos with 5 captions
Found 6 videos with 6 captions
Found 7 videos with 7 captions
Found 8 videos with 8 captions
Found 9 videos with 9 captions
Found 10 videos with 10 captions
Processed 10/608 (1.644737%) lines
Found 11 videos with 11 captions
Found 12 videos with 12 captions
Found 13 videos with 13 captions
Found 14 videos with 14 captions
Found 15 videos with 15 captions
Found 16 videos with 16 captions
Found 17 videos with 17 captions
Found 18 videos with 18 captions
Found 19 videos with 19 captions
Found 20 vide

NameError: global name 'vocab_inds_to_sentence' is not defined

In [182]:
vocab_inds_to_sentence(vocab_list, outputs['--u_-w9WCG8_14'][0]['caption'])

'<en_unk> <en_unk> <en_unk> the <en_unk> <en_unk> <en_unk> effective <en_unk> <en_unk> <en_unk>.'